# Dimensionality reduction of degree and DWPC features

# WARNING: exploratory, not functional

In [1]:
library(magrittr)

In [2]:
feature_df = readr::read_tsv('data/matrix/rephetio-v2.0/transformed-features.tsv.bz2')

In [3]:
head(feature_df, 2)

,compound_id,disease_id,status,disease_name,compound_name,prior_logit,degree_CbG,degree_CcSE,degree_CdG,degree_CiPC,ellip.h,rdwpc_CuGuDtCtD,dwpc_CuGuDuGaD,pdwpc_CuGuDuGaD,rdwpc_CuGuDuGaD,dwpc_CuGuDuGdD,pdwpc_CuGuDuGdD,rdwpc_CuGuDuGdD,dwpc_CuGuDuGuD,pdwpc_CuGuDuGuD,rdwpc_CuGuDuGuD
1,DB00091,DOID:0060073,0,lymphatic system cancer,Cyclosporine,-3.0002,4.0946,6.5338,6.2146,1.4436,⋯,1.0169,1.2737,0.88981,0.38386,0,0,0,0,0,0
2,DB00206,DOID:0060073,0,lymphatic system cancer,Reserpine,-4.4925,2.7765,4.4545,5.3376,0.88137,⋯,-0.23221,0.49511,0.31749,0.17762,0,0,0,0,0,0


In [5]:
X_df = feature_df %>%
  dplyr::select(starts_with('degree_'), starts_with('dwpc_'))

In [11]:
pca = prcomp(X_df, scale.=TRUE)

In [24]:
aurocs = predict(pca)[, 1:100] %>% apply(MARGIN = 2, FUN = function(x) {
    hetior::calc_vtms(y_true = feature_df$status, y_pred = x, is_prob = T)$auroc
})

aurocs = pmax(aurocs, 1 - aurocs)
sort(aurocs, decreasing = T)[1:10]

PC13      PC10       PC9       PC1       PC4      PC17      PC21      PC36 
0.7582352 0.6852445 0.6740525 0.6436810 0.6363857 0.6344546 0.6227560 0.5965830 
      PC6       PC8 
0.5845187 0.5759212

In [32]:
pca$rotation[, 'PC4'] %>% sort(decreasing = T) %>% head

dwpc_CbGeAeGaD dwpc_CbGeAuGaD dwpc_CbGdAeGaD dwpc_CbGeAdGaD dwpc_CbGdAdGaD 
    0.08440185     0.08385195     0.08365811     0.08353546     0.08327269 
dwpc_CbGdAuGaD 
    0.08319293

In [ ]:
aurocs

In [15]:
pc_df = as.data.frame(predict(pca)[, 1:10]) %>%
  dplyr::mutate(status = feature_df$status)

,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10,status
1,5.85976414935247,9.1854708444987,2.02842423888691,-4.28505208803047,-2.06721444570351,1.55941401836145,-0.294082022703706,-4.63582519858108,0.41845225643677,-11.0350003117416,0
2,11.3177493769339,3.60637131660705,2.088151625369,-7.94328716619681,-2.19617194254091,0.374679234829507,-1.03097105289355,-2.52101108334427,1.7488933714811,-6.63990922412877,0
3,15.1223564438297,-1.19441915258356,0.0710409251921576,-6.29384521590973,-1.78115348871976,-0.429159099640222,0.627630261913354,-1.57495990248683,-0.599901185103364,-2.19569521707825,0
4,16.5625277884498,-2.97895185528359,-0.723538459077404,-6.10932406171499,-2.84815442630085,0.877701294862815,0.646000015064222,-0.236536778261973,-0.930797300233007,-0.881984487611652,0
5,16.233344634356,-2.24460996124551,2.12441180583405,-5.83638852885673,0.149140320813077,-2.11371785005723,1.76696408657716,-1.72860569901348,-1.16594480569054,-2.83736771586684,1
6,14.753780246911,-1.27625932102355,3.4765644211243,-1.32939041741676,-0.341194337872586,0.535601114395791,1.74737343327134,-0.363512200100012,-3.04754909401439,-4.43739076541105,1
7,15.0533846393743,-2.30751028755902,-7.62021415250247,-4.88817132028559,-2.02171762347197,0.93270788231051,2.2638090104565,-0.0418067367747767,-3.60204948783068,1.24946622777255,0
8,15.9727311785879,-2.28050218452658,-1.04324652947414,-6.48208824760363,-2.67980164637912,0.140910085270906,-0.302058816426475,-0.463464740228087,-0.746239026179018,-1.0482591081263,0
9,16.2642829913751,-2.010663008236,2.64742309861245,-7.49877398147246,-2.06294940425702,0.0321633423521806,0.266871168524576,-1.16968635898241,0.750349549005404,-2.52369140651579,0
10,15.6285527881565,-1.43897531233202,2.55593772683087,-6.58047112795065,-1.5191555843051,-0.354496882530663,0.301507557395623,-1.25622532953422,0.265538615378094,-3.0777584654442,0


In [12]:
summary(pca)

Importance of components:
                           PC1     PC2     PC3     PC4     PC5     PC6     PC7
Standard deviation     19.9506 11.2083 9.12684 7.79825 5.97865 5.71685 5.37207
Proportion of Variance  0.3257  0.1028 0.06817 0.04976 0.02925 0.02675 0.02362
Cumulative Proportion   0.3257  0.4285 0.49669 0.54645 0.57570 0.60245 0.62606
                           PC8     PC9    PC10    PC11    PC12    PC13    PC14
Standard deviation     4.64398 4.40053 3.95139 3.79506 3.43632 3.27626 3.11783
Proportion of Variance 0.01765 0.01585 0.01278 0.01179 0.00966 0.00878 0.00795
Cumulative Proportion  0.64371 0.65956 0.67234 0.68412 0.69378 0.70257 0.71052
                         PC15   PC16    PC17    PC18   PC19    PC20    PC21
Standard deviation     2.9459 2.7974 2.73592 2.62822 2.4724 2.41318 2.38912
Proportion of Variance 0.0071 0.0064 0.00613 0.00565 0.0050 0.00477 0.00467
Cumulative Proportion  0.7176 0.7240 0.73015 0.73581 0.7408 0.74557 0.75025
                          PC22    PC23

prcomp {stats},R Documentation
formula,"a formula with no response variable, referring only to numeric variables."
data,an optional data frame (or similar: see model.frame) containing the variables in the formula formula. By default the variables are taken from environment(formula).
subset,an optional vector used to select rows (observations) of the data matrix x.
na.action,"a function which indicates what should happen when the data contain NAs. The default is set by the na.action setting of options, and is na.fail if that is unset. The ‘factory-fresh’ default is na.omit."
...,arguments passed to or from other methods. If x is a formula one might specify scale. or tol.
x,a numeric or complex matrix (or data frame) which provides the data for the principal components analysis.
retx,a logical value indicating whether the rotated variables should be returned.
center,"a logical value indicating whether the variables should be shifted to be zero centered. Alternately, a vector of length equal the number of columns of x can be supplied. The value is passed to scale."
scale.,"a logical value indicating whether the variables should be scaled to have unit variance before the analysis takes place. The default is FALSE for consistency with S, but in general scaling is advisable. Alternatively, a vector of length equal the number of columns of x can be supplied. The value is passed to scale."
tol,"a value indicating the magnitude below which components should be omitted. (Components are omitted if their standard deviations are less than or equal to tol times the standard deviation of the first component.) With the default null setting, no components are omitted. Other settings for tol could be tol = 0 or tol = sqrt(.Machine$double.eps), which would omit essentially constant components."
